In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.metrics import roc_auc_score
import os


with open('../datasets/mimic2/mimic2.attr') as f:
    featureNames = []
    for line in f.readlines():
        featureNames.append(line[:line.find(':')])
        
with open('../datasets/mimic2/mimic2_cont_missing.data') as f:
    data_orig = pd.read_csv(f,' ',header=None, na_values='N/A')
with open('../datasets/mimic2/mimic2_cont_rfimputed.data') as f:
    data_rf = pd.read_csv(f,' ',header=None, na_values='N/A')
# with open('../datasets/mimic2/mimic2_cont_knnimputed.data') as f:
#     data_knn = pd.read_csv(f,' ',header=None, na_values='N/A')
    
data_orig.columns = featureNames
# data_mean.columns = featureNames
data_rf.columns = featureNames
# data_knn.columns = featureNames

In [2]:
for feature in ['AdmissionType', 'AIDS', 'MetastaticCancer' , 'Lymphoma', 'HospitalMortality']:
    data_rf[feature] = data_rf[feature].astype('int64')

In [3]:
is_missing = data_orig.isnull().sum()>0
print(is_missing)

Age                  False
GCS                  False
SBP                   True
HR                    True
Temperature          False
PFratio               True
Renal                False
Urea                  True
WBC                   True
CO2                   True
Na                    True
K                     True
Bilirubin             True
AdmissionType        False
AIDS                 False
MetastaticCancer     False
Lymphoma             False
HospitalMortality    False
dtype: bool


In [36]:
target_feature = 'Na'
train_cols = data_rf.columns
X_rf = data_rf[train_cols].copy()
X_rf = X_rf.drop(columns=[target_feature])
for i, featureName in enumerate(X_rf.columns):
    X_rf[featureName][data_orig[featureName].isnull()] = X_rf[featureName][data_orig[featureName].isnull()] + data_orig[featureName].max() + 1

y_missing = data_orig.isnull().astype('int64')
y_missing_target = y_missing[target_feature]
seed=1
X_train_rf, X_test_rf, y_train_missing, y_test_missing = train_test_split(X_rf, y_missing_target, test_size=0.2, random_state=seed)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
ebm = ExplainableBoostingClassifier(outer_bags = 20, inner_bags= 20, interactions=0,random_state=seed, n_jobs=-1)
ebm.fit(X_train_rf, y_train_missing)   #Works on dataframes and numpy arrays
y_pred = ebm.predict_proba(X_test_rf)[:,1]
print(f'AUC = {roc_auc_score(y_test_missing,y_pred)}')
show(ebm.explain_global())

AUC = 0.9987487487487487


<!-- http://127.0.0.1:7585/112651067632/ -->

In [8]:
ebm.feature_importances_

[0.1281317494217416,
 0.3091267663338043,
 0.19855159030763017,
 0.11851986653332242,
 0.4870254696771734,
 0.29337401234587135,
 0.17854843095554,
 0.025728864441840978,
 0.038089040072783596,
 0.26939501478715056,
 0.04031441121883893,
 0.530264385341978,
 0.19930583031342727,
 0.02271689451701991,
 0.028809483119970843,
 0.034554147651751584,
 0.12128913758819831]

In [11]:
data_orig.columns[np.argmax(ebm.feature_importances_)]

'K'

In [18]:
data_rf.max(0)

Age                   105.341547
GCS                     5.000000
SBP                   289.000000
HR                    216.000000
Temperature             7.000000
PFratio              2647.619048
Renal                   4.000000
Urea                  274.000000
WBC                   846.700000
CO2                    52.000000
Na                    176.000000
K                      17.100000
Bilirubin              82.800000
AdmissionType           2.000000
AIDS                    1.000000
MetastaticCancer        1.000000
Lymphoma                1.000000
HospitalMortality       1.000000
dtype: float64

In [11]:
def plot_seperated_shape_functions(ebm, data_orig, data_rf, target_feature):
    featureNames = ebm.feature_names
    is_missing  = data_orig.isnull().sum()>0
    max_feature_values = data_orig.max(0)
    min_feature_values = data_orig.min(0)
    offsets = max_feature_values+1
    max_feature_values['Temperature']=6
    
    max_score = -10
    min_score = 10
    for i, featureName in enumerate(featureNames):
        max_score = max(max_score, ebm.additive_terms_[i].max())
        min_score = min(min_score, ebm.additive_terms_[i].min())
    
    for i, featureName in enumerate(featureNames):
        if ebm.feature_types[i]=='continuous':
            # --------get the scores and bins of the shape function--------
            scores = ebm.additive_terms_[i]
            scores[0] = scores[1]
            bins = np.zeros(scores.shape)
            if bins.shape[0]>2:
                bins[1:-1] = ebm.preprocessor_.col_bin_edges_[i]
            bins[0] = ebm.preprocessor_.col_min_[i]
            bins[-1] = ebm.preprocessor_.col_max_[i]
            if is_missing[featureName]>0:
                # part 1: shape function for the observed feature values
                idx_max = np.argmax(bins>max_feature_values[featureName])
                bins_observed = bins[:idx_max+1].copy()
                bins_observed[-1] = max_feature_values[featureName]
                scores_observed = scores[:idx_max+1].copy()
                # part 2 shape function for the imputed feature values
                min_imputed_value = data_rf[featureName][data_orig[featureName].isnull()].min()
                idx_imputed = np.argmax(bins>min_imputed_value) - 1
                bins_imputed = bins[idx_imputed:].copy()
                bins_imputed[0] = min_imputed_value
                scores_imputed = scores[idx_imputed:].copy()
                scores_imputed[0] = scores_imputed[1]
            else:
                bins_observed,  scores_observed = bins, scores
            # --------plot the shape function--------
            plt.figure(figsize=(10,6))
            gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1]) 
            plt.subplot(gs[0])
            plt.step(bins_observed,scores_observed,label='pre (default)')# observed feature values
            if is_missing[featureName]>0:
                plt.step(bins_imputed-max_feature_values[featureName]-1,scores_imputed,label='pre (default)')# imputed feature values 
            if target_feature=='Na' and featureName=='Bilirubin':
                plt.xlim([min_feature_values[featureName], 10])
            else:
                plt.xlim([min_feature_values[featureName], max_feature_values[featureName]])
#             plt.ylim([min_score,max_score])
            plt.xlabel(featureNames[i],fontsize=14)
            if target_feature=='Na':
                plt.ylabel(f'{target_feature} (Sodium) is missing',fontsize=14)
            else:
                plt.ylabel(f'{target_feature} is missing',fontsize=14)
            if is_missing[featureName]>0:
                plt.legend(['observed (non missing)','imputed (RF)'])


            # --------plot the density--------
#             plt.subplot(gs[1])
# #             plt.hist(data_rf.values[:,i],bins_observed,alpha=.7,density=True)# observed feature values
#             plt.hist(data_rf.values[:,i],bins=np.linspace(min_feature_values[featureName], max_feature_values[featureName],100),alpha=.7)
#             if is_missing[featureName]>0:
# #                 plt.hist(data_rf.values[:,i]-offsets[featureName],bins_imputed-offsets[featureName],alpha=.7,density=True)
#                 plt.hist(data_rf.values[:,i]-offsets[featureName],bins=np.linspace(min_feature_values[featureName], max_feature_values[featureName],100),alpha=.7)
#             if target_feature=='Na' and featureName=='Bilirubin':
#                 plt.xlim([min_feature_values[featureName], 10])
#             else:
#                 plt.xlim([min_feature_values[featureName], max_feature_values[featureName]])
#             plt.xlabel(featureNames[i],fontsize=14)
#             plt.ylabel('density',fontsize=14)
#             if is_missing[featureName]>0:
#                 plt.legend(['observed (non missing)','imputed (RF)'])
            folder = f"mimic2_predicting_missingness/{target_feature}"
            os.makedirs(folder,exist_ok=True)
            plt.savefig(f'{folder}/{i}_{featureNames[i]}.png', dpi=300)

In [9]:
def predicting_missingness(target_feature):
    train_cols = data_rf.columns
    X_rf = data_rf[train_cols].copy()
    X_rf = X_rf.drop(columns=[target_feature])
    for i, featureName in enumerate(X_rf.columns):
        X_rf[featureName][data_orig[featureName].isnull()] = X_rf[featureName][data_orig[featureName].isnull()] + data_orig[featureName].max() + 1

    y_missing = data_orig.isnull().astype('int64')
    y_missing_target = y_missing[target_feature]
    seed=1
    X_train_rf, X_test_rf, y_train_missing, y_test_missing = train_test_split(X_rf, y_missing_target, test_size=0.2, random_state=seed)
    ebm = ExplainableBoostingClassifier(outer_bags = 20, inner_bags= 20, interactions=0,random_state=seed, n_jobs=-1)
    ebm.fit(X_train_rf, y_train_missing)   #Works on dataframes and numpy arrays
    y_pred = ebm.predict_proba(X_test_rf)[:,1]
    print(f'missing feature: {target_feature}, AUC = {roc_auc_score(y_test_missing,y_pred)}')
    print(f'most important feature {X_rf.columns[np.argmax(ebm.feature_importances_)]}')
    
    plot_seperated_shape_functions(ebm,data_orig,X_train_rf,target_feature)

In [12]:
predicting_missingness('Na')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


missing feature: Na, AUC = 0.9987487487487487
most important feature Bilirubin


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [6]:
import matplotlib
matplotlib.use('Agg')

In [10]:
for featureName in featureNames:
    if is_missing[featureName]:
        predicting_missingness(featureName)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


missing feature: SBP, AUC = 0.863935059361555
most important feature GCS


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


missing feature: HR, AUC = 0.875024633554639
most important feature SBP


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


...

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


missing feature: K, AUC = 0.8602579852579852
most important feature Temperature


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


missing feature: Bilirubin, AUC = 0.7355658522912603
most important feature Urea


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
